In [1]:
% load_ext autoreload
% autoreload 2

% matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
data_path = 'data/bulldozers/'

In [4]:
bd_raw_train = pd.read_csv(f'{data_path}Train.csv'
                          , low_memory = False
                          , parse_dates = ['saledate']
                          , index_col = 'SalesID')

In [5]:
print(f'The shape of the train set is {bd_raw_train.shape}')


The shape of the train set is (401125, 52)


In [6]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

display_all(bd_raw_train.tail().transpose())

SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00
fiModelDesc,35NX2,35NX2,35NX2,30NX,30NX


In [7]:
allMissing = bd_raw_train.isnull().sum().sort_values(ascending=False)
percentage = (bd_raw_train.isnull().sum()/bd_raw_train.isnull().count()).sort_values(ascending=False)
missingData = pd.concat([allMissing, percentage*100], axis=1, keys=['TotalMissing', 'Percentage'])
missingData.head(40)

,TotalMissing,Percentage
Tip_Control,375906,93.712932
Blade_Extension,375906,93.712932
Blade_Width,375906,93.712932
Enclosure_Type,375906,93.712932
Engine_Horsepower,375906,93.712932
Pushblock,375906,93.712932
Scarifier,375895,93.710190
Hydraulics_Flow,357763,89.189903
Grouser_Tracks,357763,89.189903
Coupler_System,357667,89.165971


In [8]:
bd_raw_train = bd_raw_train.drop(
    labels = list(allMissing.index)[:21]
    , axis = 1)
display_all(bd_raw_train.tail().transpose())

SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00
fiModelDesc,35NX2,35NX2,35NX2,30NX,30NX
fiBaseModel,35,35,35,30,30


In [9]:
cols_drop = ['MachineID', 'ModelID']
bd_raw_train = bd_raw_train.drop(labels = cols_drop
                                , axis = 1)
bd_raw_train['datasource'] = bd_raw_train['datasource'].apply(str)
bd_raw_train['auctioneerID'] = bd_raw_train['auctioneerID'].apply(str)

In [10]:
bd_raw_train = bd_raw_train.drop(labels = ['MachineHoursCurrentMeter']
                                , axis = 1)

In [11]:
bd_raw_train['Tire_Size'] = bd_raw_train['Tire_Size'].fillna('None or Unspecified')
bd_raw_train['Grouser_Type'] = bd_raw_train['Grouser_Type'].fillna('na')
bd_raw_train['Track_Type'] = bd_raw_train['Track_Type'].fillna('na')
bd_raw_train['Stick_Length'] = bd_raw_train['Stick_Length'].fillna('None or Unspecified')
bd_raw_train['Pattern_Changer'] = bd_raw_train['Pattern_Changer'].fillna('None or Unspecified')
bd_raw_train['Thumb'] = bd_raw_train['Thumb'].fillna('None or Unspecified')
bd_raw_train['Undercarriage_Pad_Width'] = bd_raw_train['Undercarriage_Pad_Width'].fillna('None or Unspecified')
bd_raw_train['Ripper'] = bd_raw_train['Ripper'].fillna('None or Unspecified')
bd_raw_train['Drive_System'] = bd_raw_train['Drive_System'].fillna('na')
bd_raw_train['Ride_Control'] = bd_raw_train['Ride_Control'].fillna('None or Unspecified')
bd_raw_train['Transmission'] = bd_raw_train['Transmission'].fillna('None or Unspecified')
bd_raw_train['ProductSize'] = bd_raw_train['ProductSize'].fillna('na')
bd_raw_train['Forks'] = bd_raw_train['Forks'].fillna('None or Unspecified')
bd_raw_train['Coupler'] = bd_raw_train['Coupler'].fillna('None or Unspecified')
bd_raw_train['fiSecondaryDesc'] = bd_raw_train['fiSecondaryDesc'].fillna('na')
bd_raw_train['Hydraulics'] = bd_raw_train['Hydraulics'].fillna('None or Unspecified')
bd_raw_train['Enclosure'] = bd_raw_train['Enclosure'].fillna('None or Unspecified')

In [12]:
bd_raw_train['saleyear'] = bd_raw_train['saledate'].dt.year
bd_raw_train['salemonth'] = bd_raw_train['saledate'].dt.month
bd_raw_train['saleday'] = bd_raw_train['saledate'].dt.day
bd_raw_train['saleweekday'] = bd_raw_train['saledate'].dt.dayofweek
bd_raw_train['salequarter'] = bd_raw_train['saledate'].dt.quarter
bd_raw_train['saledoy'] = bd_raw_train['saledate'].dt.dayofyear
bd_raw_train = bd_raw_train.drop(labels = ['saledate'], axis = 1)

In [13]:
X_train = pd.get_dummies(bd_raw_train.drop(labels = ['SalePrice', 'fiModelDesc', 'fiBaseModel'], axis = 1)
                         , drop_first = True)
y_train = bd_raw_train['SalePrice']

In [14]:
X_train.shape

(401125, 463)